# Welcome to the Assembly Lesson

In this lesson you will learn about programming ARM assembly by stepping through example programs and completing a few assembly programming assignments.

## Example: Hello World in Assembly

In [28]:
%%file hello_world.c

#include <stdio.h>

int main(void) {
  printf("Hello, world.\n");
  return 0;
}

Overwriting hello_world.c


In [13]:
%%file hello_world.s
.syntax unified

    @ --------------------------------
.global main
    
main:
    @ Stack the return address (lr) in addition to a dummy register (ip) to
    @ keep the stack 8-byte aligned.
    push    {ip, lr}

    @ Load the argument and perform the call. This is like 'printf("...")' in C.
    ldr     r0, =message
    bl      printf

    @ Exit from 'main'. This is like 'return 0' in C.
    mov     r0, #0    @ Return 0.

    @ Pop the dummy ip to reverse our alignment fix, and pop the original lr
    @ value directly into pc — the Program Counter — to return.
    pop     {ip, pc}

    @ --------------------------------
    @ Data for the printf calls. The GNU assembler's ".asciz" directive
    @ automatically adds a NULL character termination.

message:
    .asciz "Hello, world.\n"


Overwriting hello_world.s


In [14]:
!arm-linux-gnueabi-gcc -o hello_world hello_world.s

In [17]:
!QEMU_LD_PREFIX=/usr/arm-linux-gnueabi ./hello_world

Hello, world.


Let's debug this program and step through each instruction. First, start the hello world program with the -g flag to indicate that you would like to debug the program with gdb. This will start gdb server which we can connect to in another terminal window to start our debugging:

    qemu-arm -g 12345 ./hello_world
    
In another window start gdb and connect to the gdb server with the following commands:

    $ gdb-multiarch
    GNU gdb (Ubuntu 8.1-0ubuntu3.2) 8.1.0.20180409-git
    Copyright (C) 2018 Free Software Foundation, Inc.
    License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html&gt;
    This is free software: you are free to change and redistribute it.
    There is NO WARRANTY, to the extent permitted by law.  Type "show copying"
    and "show warranty" for details.
    This GDB was configured as "x86_64-linux-gnu".
    Type "show configuration" for configuration details.
    For bug reporting instructions, please see:
    <http://www.gnu.org/software/gdb/bugs/&gt;.
    Find the GDB manual and other documentation resources online at:
    <http://www.gnu.org/software/gdb/documentation/&gt;.
    For help, type "help".
    Type "apropos word" to search for commands related to "word".
    pwndbg: loaded 180 commands. Type pwndbg [filter] for a list.
    pwndbg: created $rebase, $ida gdb functions (can be used with print/break)
    pwndbg> target remote :12345
    

In [18]:
!QEMU_LD_PREFIX=/usr/arm-linux-gnueabi qemu-arm -g 12345 
./hello_world

^C


## Example: Subroutines (or functions) in Assembly

In [29]:
%%file subroutine.c
#include <stdio.h>

void subroutine() {
    printf("Hello, world.\n");
}

int main(void) {
    subroutine();
    return 0;
}


Overwriting subroutine.c


In [30]:
%%file subroutine.s
.syntax unified

    @ --------------------------------
.global main
    
main:
    @ Stack the return address (lr) in addition to a dummy register (ip) to
    @ keep the stack 8-byte aligned.
    push    {ip, lr}

    @ Branch and link to subroutine
    bl     subroutine

    @ Exit from 'main'. This is like 'return 0' in C.
    mov     r0, #0    @ Return 0.

    @ Pop the dummy ip to reverse our alignment fix, and pop the original lr
    @ value directly into pc — the Program Counter — to return.
    pop     {ip, pc}

    @ --------------------------------
    @ Data for the printf calls. The GNU assembler's ".asciz" directive
    @ automatically adds a NULL character termination.

.global subroutine

subroutine:
    @ Stack the return address (lr) in addition to a dummy register (ip) to
    @ keep the stack 8-byte aligned.
    push    {ip, lr}
    
    @ Load the argument and perform the call. This is like 'printf("...")' in C.
    ldr     r0, =message
    bl      printf

    @ Pop the dummy ip to reverse our alignment fix, and pop the original lr
    @ value directly into pc — the Program Counter — to return.
    pop     {ip, pc}
    
message:
    .asciz "Hello, world.\n"


Overwriting subroutine.s


In [31]:
!arm-linux-gnueabi-gcc -o subroutine subroutine.s

In [27]:
!QEMU_LD_PREFIX=/usr/arm-linux-gnueabi ./subroutine

Hello, world.
